In [66]:
# load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings as wr
wr.filterwarnings('ignore')
import os

In [67]:
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,LSTM,GRU,Embedding , SpatialDropout1D
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, classification_report, confusion_matrix
from tensorflow.keras.datasets import imdb
from keras.preprocessing import sequence


In [68]:
# load the IMDB dataset
vocab_size= 10000
max_words = 500
(X_train,y_train),(X_test,y_test) = imdb.load_data(num_words=vocab_size)
# Load the IMDB dataset


Exception ignored in: <function _xla_gc_callback at 0x7a3925fd75b0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 98, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


In [9]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((25000,), (25000,), (25000,), (25000,))

In [32]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = vocab_size)

In [18]:
# Convert integer sequences back to strings
X_train = [str(x) for x in X_train]
X_test = [str(x) for x in X_test]

X_train = tokenizer.texts_to_sequences(X_train)
X_test  = tokenizer.texts_to_sequences(X_test)
# pad the sequence
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)


In [21]:
X_train.shape, X_test.shape

((25000, 500), (25000, 500))

In [22]:
y_train.shape,y_test.shape

((25000,), (25000,))

In [37]:
# Define the LSTM Model
model = Sequential()
model.add(Embedding(vocab_size, 128, input_length=max_words))
SpatialDropout1D(0.4)
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(64, activation = 'relu'))
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 500, 128)          1280000   
                                                                 
 lstm_2 (LSTM)               (None, 128)               131584    
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dense_5 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1419905 (5.42 MB)
Trainable params: 1419905 (5.42 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
plot_model(model, show_shapes=True)


In [39]:
# compile the model
model.compile(loss='binary_crossentropy', optimizer = 'adam' , metrics = ['accuracy'])

In [51]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Pad sequences to ensure uniform length
X_train = pad_sequences(X_train, maxlen=max_words)

# Convert lists to NumPy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)

# Additional checks
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_train data type:", X_train.dtype)
print("y_train data type:", y_train.dtype)

# Train the model
# model.fit(X_train, y_train, epochs=5, batch_size=64, steps_per_epoch=len(X_train) // 64)
history=model.fit(X_train, y_train, epochs=5, batch_size=64, steps_per_epoch=20)


X_train shape: (25000, 500)
y_train shape: (25000,)
X_train data type: int32
y_train data type: int64
Epoch 1/5
20/20 [==============================] - 41s 2s/step - loss: 0.4095 - accuracy: 0.8125
Epoch 2/5
20/20 [==============================] - 75s 4s/step - loss: 0.3590 - accuracy: 0.8539
Epoch 3/5
20/20 [==============================] - 45s 2s/step - loss: 0.8353 - accuracy: 0.7484
Epoch 4/5
20/20 [==============================] - 47s 2s/step - loss: 0.4977 - accuracy: 0.8039
Epoch 5/5
20/20 [==============================] - 46s 2s/step - loss: 0.4719 - accuracy: 0.8398


In [53]:

train_accuracy = model.evaluate(X_train, y_train)[1] * 100
print("Train Accuracy:", train_accuracy, "%")


782/782 [==============================] - 184s 234ms/step - loss: 0.4184 - accuracy: 0.8429
Train Accuracy: 84.29200053215027 %


In [62]:
# Get the loss and accuracy values from the history object
loss = history.history['loss']
accuracy = history.history['accuracy']

# Plot the loss over epochs
plt.plot(loss, label='Loss')
plt.title('Loss over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Plot the accuracy over epochs
plt.plot(accuracy, label='Accuracy')
plt.title('Accuracy over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


Epoch 1/5
 3/20 [===>..........................] - ETA: 45s - loss: 0.4543 - accuracy: 0.8333

KeyboardInterrupt: 

In [63]:

from sklearn.metrics import confusion_matrix
import seaborn as sns

# Predict the labels for the test data
y_pred = model.predict(X_test)

# Convert the predictions to binary values (0 or 1)
y_pred_binary = [int(round(p[0])) for p in y_pred]

# Generate the confusion matrix
cm = confusion_matrix(y_test, y_pred_binary)

# Create a heatmap of the confusion matrix
sns.heatmap(cm, annot=True, cmap="Blues", fmt="d")
plt.xlabel("Predicted label")
plt.ylabel("True label")
plt.title("Confusion Matrix")
plt.show()


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type list).

In [56]:
# Print the classification report
print(classification_report(y_test, y_pred_binary))

NameError: name 'y_pred_binary' is not defined

In [69]:
def predict_sentiment(sentence):
  # Convert the sentence to a sequence of integers
  sentence = tokenizer.texts_to_sequences([sentence])
  sentence = pad_sequences(sentence, maxlen=max_words)

  # Predict the sentiment
  prediction = model.predict(sentence)

  # Map the prediction to a sentiment category
  if prediction[0] > 0.5:
    sentiment = "Positive"
  elif prediction[0] < 0.5:
    sentiment = "Negative"
  else:
    sentiment = "Neutral"

  # Return the predicted sentiment
  return sentiment

# Get user input
sentence = input("Enter a sentence: ")

# Predict the sentiment
sentiment = predict_sentiment(sentence)

# Print the predicted sentiment
print(f"Predicted sentiment: {sentiment}")


Enter a sentence: Its a very good movie.Everyone should have watch this movie.
1/1 [==============================] - 0s 191ms/step
Predicted sentiment: Positive
